In [1]:
import xnetwork as xn
from igraph import *
import matplotlib.pyplot as plt

# dic[subject] = community
# Community 0: fiction
# Community 1: juvenile fiction
# essas duas sao bem concentradas e definidinhas, o restante ja era bem mais espalhado, dai nao 
# estudei muito oq cada uma quer dizer

# coloquei o padrao pra 10 comunidades, pq me falaram pra fazer assim ha um tempo atras kkk mas acabamos mudando pra 3
# ai se vc chamar a funcao com 3 vai acabar ficando 0=ficcao, 1=ficcao juvenil e 2=outros
# e inclusive pra mim depois acabamos juntando 0 com 1 e ficando so com "ficcao generico" ou "outros" haha
# dai vc ve oq functiona melhor pra vc hahah qualquer coisa me avisa :)
def get_community_labels(n_comm=10):

    G = xn.xnet2igraph('projection_onto_subjects_comm.xnet')

    dic_subj_comm = {}

    for v in G.vs:
        dic_subj_comm[v['name']] = int(v['Community']) if int(v['Community']) < n_comm - 1 else n_comm - 1

    return dic_subj_comm

In [14]:
from collections import Counter
from gutenberg.query import get_metadata

def retrieve_subjects(index):
    return list(get_metadata('subject', index))

def format_subject(subject):
    return ''.join(c for c in subject if c.lower() in 'abcdefghijklmnopqrstuvwxyz1234567890 -')

def get_book_community(dic_subj_comm, book_id):
    subjects = [format_subject(subj) for subj in retrieve_subjects(book_id)]
    targets = [dic_subj_comm[subj] for subj in subjects]

    most_common = Counter(targets).most_common(1)
    if len(most_common) > 0 and (most_common[0][0] == 0 or most_common[0][0] == 1):
        return 0
    return 1

In [15]:
# book_id de alice no pais das maravilhas: 11
dic = get_community_labels()
get_book_community(dic, 11)

0

In [11]:
Counter(labels).most_common(1)

[(0, 111)]

In [5]:
import os

ids = []
path = './pca 300 networks new right/'
directory = os.fsencode(path)

for file in os.listdir(directory):
    filename = path + os.fsdecode(file)
    if filename.endswith("_RP.tsv"):
        start = len(path) + len('network_')
        end = len('_RP.tsv')
        gutenberg_id = filename[start:-end]
        ids.append(gutenberg_id)

In [16]:
labels = []
for id in ids:
    labels.append(get_book_community(dic, id))
    
Counter(labels)

Counter({0: 182, 1: 118})

In [19]:
import pandas as pd

df = pd.DataFrame()
df['gutenberg_id'] = ids
df['label'] = labels

df.to_csv('genre_labels.csv', index=False)